<a href="https://colab.research.google.com/github/ckkhandare/DS_firstProject/blob/main/DSProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Libraries and loading Data

Internshala Project

In [ ]:
! pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip

     \ 34.6 MB 856 kB/s
     |████████████████████████████████| 10.1 MB 5.3 MB/s 
     |████████████████████████████████| 636 kB 38.6 MB/s 
     |████████████████████████████████| 102 kB 50.7 MB/s 
     |████████████████████████████████| 675 kB 42.2 MB/s 
     |████████████████████████████████| 3.1 MB 42.0 MB/s 
     |████████████████████████████████| 62 kB 819 kB/s 
     |████████████████████████████████| 812 kB 51.6 MB/s 
     |████████████████████████████████| 28.5 MB 50 kB/s 
  Created wheel for pandas-profiling: filename=pandas_profiling-3.0.0-py2.py3-none-any.whl size=248524 sha256=026b0d61afd838844436b4e3e90b04de4370c6d739916fe6e64201c27cbd9a94
  Stored in directory: /tmp/pip-ephem-wheel-cache-gp6a989q/wheels/cc/d5/09/083fb07c9363a2f45854b0e3a7de7d7c560f07da74b9e9769d
  Created wheel for htmlmin: filename=htmlmin-0.1.12-py3-none-any.whl size=27098 sha256=4c94a53a693e85c995c0ccfbfc26cdfc5c8d938b290068170193f76079006575
  Stored in directory: /root/.cache/pip/wheels/70/e1/52/5b14d

In [ ]:
! pip install catboost

     |████████████████████████████████| 67.4 MB 28 kB/s 


In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder,OneHotEncoder,OrdinalEncoder,RobustScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split,GridSearchCV,RandomizedSearchCV,cross_val_score,StratifiedKFold
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from pandas_profiling import ProfileReport
from pickle import dump

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [46]:
df=pd.read_csv('train.csv',na_values='unknown')

# Exploratory Data Analysis

In [47]:
df.head(5)

,ID,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,subscribed
0,26110,56,admin.,married,NaN,no,1933,no,no,telephone,19,nov,44,2,-1,0,NaN,no
1,40576,31,NaN,married,secondary,no,3,no,no,cellular,20,jul,91,2,-1,0,NaN,no
2,15320,27,services,married,secondary,no,891,yes,no,cellular,18,jul,240,1,-1,0,NaN,no
3,43962,57,management,divorced,tertiary,no,3287,no,no,cellular,22,jun,867,1,84,3,success,yes
4,29842,31,technician,married,secondary,no,119,yes,no,cellular,4,feb,380,1,-1,0,NaN,no


In [ ]:
profile=ProfileReport(df,title="Profiling Report",minimal=True)

In [ ]:
profile.to_file("EDA_report.html")

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [48]:
df.shape

(31647, 18)

In [49]:
df.isnull().sum()*100/31647

ID             0.000000
age            0.000000
job            0.650931
marital        0.000000
education      4.152052
default        0.000000
balance        0.000000
housing        0.000000
loan           0.000000
contact       28.998009
day            0.000000
month          0.000000
duration       0.000000
campaign       0.000000
pdays          0.000000
previous       0.000000
poutcome      81.931937
subscribed     0.000000
dtype: float64

In [50]:
df.select_dtypes(include='O').columns

Index(['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact',
       'month', 'poutcome', 'subscribed'],
      dtype='object')

In [51]:
df.select_dtypes(exclude='O').columns

Index(['ID', 'age', 'balance', 'day', 'duration', 'campaign', 'pdays',
       'previous'],
      dtype='object')

In [52]:
df.describe()

,ID,age,balance,day,duration,campaign,pdays,previous
count,31647.000000,31647.000000,31647.000000,31647.000000,31647.000000,31647.000000,31647.000000,31647.000000
mean,22563.972162,40.957247,1363.890258,15.835466,258.113534,2.765697,39.576042,0.574272
std,13075.936990,10.625134,3028.304293,8.337097,257.118973,3.113830,99.317592,2.422529
min,2.000000,18.000000,-8019.000000,1.000000,0.000000,1.000000,-1.000000,0.000000
25%,11218.000000,33.000000,73.000000,8.000000,104.000000,1.000000,-1.000000,0.000000
50%,22519.000000,39.000000,450.000000,16.000000,180.000000,2.000000,-1.000000,0.000000
75%,33879.500000,48.000000,1431.000000,21.000000,318.500000,3.000000,-1.000000,0.000000
max,45211.000000,95.000000,102127.000000,31.000000,4918.000000,63.000000,871.000000,275.000000


In [53]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31647 entries, 0 to 31646
Data columns (total 18 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   ID          31647 non-null  int64 
 1   age         31647 non-null  int64 
 2   job         31441 non-null  object
 3   marital     31647 non-null  object
 4   education   30333 non-null  object
 5   default     31647 non-null  object
 6   balance     31647 non-null  int64 
 7   housing     31647 non-null  object
 8   loan        31647 non-null  object
 9   contact     22470 non-null  object
 10  day         31647 non-null  int64 
 11  month       31647 non-null  object
 12  duration    31647 non-null  int64 
 13  campaign    31647 non-null  int64 
 14  pdays       31647 non-null  int64 
 15  previous    31647 non-null  int64 
 16  poutcome    5718 non-null   object
 17  subscribed  31647 non-null  object
dtypes: int64(8), object(10)
memory usage: 4.3+ MB


#### Observation

poutcome has more than 81.93% data missing

education is the only ordinal feature rest all as Nominal ['job','marital','default', 'housing','loan', 'contact','month', 'poutcome','subscribed']

job , education and contact have missing values that need to be imputed

previous is highly skewed and has 81% of zeros

where previous is zero pdays is -1 ,have to handle that

# Preprocessing

In [54]:
df.drop(columns=['poutcome'],inplace=True)

In [55]:
df.drop(columns=['ID'],inplace=True)

In [56]:
df["pdays"].replace({-1: 0}, inplace=True)

In [57]:
df.shape

(31647, 16)

In [58]:
df.isnull().sum()

age              0
job            206
marital          0
education     1314
default          0
balance          0
housing          0
loan             0
contact       9177
day              0
month            0
duration         0
campaign         0
pdays            0
previous         0
subscribed       0
dtype: int64

In [59]:
X=df.drop(columns=['subscribed'])
y=df['subscribed']

In [60]:
y.value_counts(normalize=True)

no     0.882611
yes    0.117389
Name: subscribed, dtype: float64

In [61]:
X.columns

Index(['age', 'job', 'marital', 'education', 'default', 'balance', 'housing',
       'loan', 'contact', 'day', 'month', 'duration', 'campaign', 'pdays',
       'previous'],
      dtype='object')

In [62]:
X.contact.unique()

array(['telephone', 'cellular', nan], dtype=object)

In [63]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.30, random_state=42,stratify=y)
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((22152, 15), (9495, 15), (22152,), (9495,))

In [64]:
le=LabelEncoder()

In [65]:
y_train1=le.fit_transform(y_train)
y_test1=le.transform(y_test)

In [66]:
pd.DataFrame(y_train1).value_counts(normalize=True)

0    0.882629
1    0.117371
dtype: float64

In [67]:
pd.DataFrame(y_test1).value_counts(normalize=True)

0    0.88257
1    0.11743
dtype: float64

Data is imbalanced

In [68]:
education=['primary','secondary','tertiary']

In [69]:
ordi=['education']
nomi=['job', 'marital', 'default',  'housing', 'loan', 'contact',
       'month']

In [70]:
#continuous
continuous=['age', 'balance', 'day', 'duration', 'campaign', 'pdays',
       'previous']

In [71]:
p_numeric=Pipeline([
                    ('StandardS',RobustScaler())
])

In [72]:
p_cat_ordi=Pipeline([
                     ('Impute_num',SimpleImputer(strategy='most_frequent')),
                ('Ordinal_encode',OrdinalEncoder(categories=[['primary','secondary','tertiary']]))
])

In [73]:
p_cat_nomi=Pipeline([
                     ('Impute_num',SimpleImputer(strategy='most_frequent')),
                     ('One_hot',OneHotEncoder(drop='first'))
])

In [74]:
col_trans=ColumnTransformer(transformers=[                                          
    ('scale',p_numeric,['age', 'balance', 'day', 'duration', 'campaign', 'pdays',
       'previous']),
    ('O_encode',p_cat_ordi,['education']),
    ('N_encode',p_cat_nomi,['job', 'marital', 'default',  'housing', 'loan', 'contact','month'])

],remainder='drop')

In [75]:
X_train1=col_trans.fit_transform(X_train)
X_test1=col_trans.transform(X_test)

In [76]:
X_train1.shape,X_test1.shape

((22152, 35), (9495, 35))

In [78]:
sm=SMOTE(random_state=42)

In [79]:
X_train_bal,y_train_bal=sm.fit_resample(X_train1,y_train1)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [80]:
pd.DataFrame(y_train_bal).value_counts(normalize=True)

1    0.5
0    0.5
dtype: float64

#Feature selection

In [ ]:
# sel_X_train=np.array(X_train_bal)
# sel_y_train=np.array(y_train)
rf =RandomForestClassifier()

In [ ]:
rf.fit(X_train_bal,y_train_bal)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
len(rf.feature_importances_)

35

In [ ]:
rf.feature_importances_

array([0.0551066 , 0.06309347, 0.05543432, 0.29541308, 0.06868495,
       0.03739113, 0.0301391 , 0.0265548 , 0.02306339, 0.00302262,
       0.00313312, 0.01366225, 0.00521723, 0.00420868, 0.00902873,
       0.00500793, 0.01124984, 0.00390181, 0.02286547, 0.01437189,
       0.00136225, 0.07253866, 0.03008731, 0.00722666, 0.01361395,
       0.00223014, 0.01149676, 0.0071408 , 0.01356333, 0.01024261,
       0.01188942, 0.03705589, 0.01123718, 0.01294474, 0.00681991])

In [ ]:
selected_col= pd.DataFrame({'columns' : pd.DataFrame(X_train_bal).columns , 'imp_fet': list(rf.feature_importances_)})

In [ ]:
selected_col.sort_values(by='imp_fet',ascending=False).head(22)

,columns,imp_fet
3,3,0.295413
21,21,0.072539
4,4,0.068685
1,1,0.063093
2,2,0.055434
0,0,0.055107
5,5,0.037391
31,31,0.037056
6,6,0.030139
22,22,0.030087


In [ ]:
sorted_sel=selected_col.sort_values(by='imp_fet',ascending=False).head(22)

In [ ]:
sorted_sel

,columns,imp_fet
3,3,0.295413
21,21,0.072539
4,4,0.068685
1,1,0.063093
2,2,0.055434
0,0,0.055107
5,5,0.037391
31,31,0.037056
6,6,0.030139
22,22,0.030087


In [ ]:
X_train_bal_sel = pd.DataFrame( pd.DataFrame(X_train_bal),columns = sorted_sel['columns'])
X_test_bal_sel = pd.DataFrame( pd.DataFrame(X_test1),columns = sorted_sel['columns'])


# model selection 

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression

from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.neural_network import MLPClassifier

In [ ]:
cl0=KNeighborsClassifier()
cl1=SVC()
cl2=GaussianNB()
cl3=LogisticRegression()
label=['KNeighborsClassifier','SVC','GaussianNB','LogisticRegression']
for clf,label in zip([cl0,cl1,cl2,cl3],label):
  score=cross_val_score(clf,X_train_bal,y_train_bal,cv=5,scoring='roc_auc',n_jobs=-1,verbose=2)
  print(score.mean(),label)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   18.2s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   18.2s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


0.9587331084878551 KNeighborsClassifier


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  4.2min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  4.2min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


0.8595145704753149 SVC


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.2s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


0.8054859796908032 GaussianNB
nan LogisticRegression


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    2.1s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    2.1s finished


In [ ]:
cl4=RandomForestClassifier()
cl5=XGBClassifier()
cl6=CatBoostClassifier()
label=['RandomForestClassifier','XGBClassifier','CatBoostClassifier']
for clf,label in zip([cl4,cl5,cl6],label):
  score=cross_val_score(clf,X_train_bal,y_train_bal,cv=5,scoring='roc_auc',n_jobs=-1,verbose=2)
  print(score.mean(),label)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   15.4s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   15.4s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


0.9921006347026843 RandomForestClassifier


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   12.9s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   12.9s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


0.9734625248465051 XGBClassifier


/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


0.9821696669533176 CatBoostClassifier


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  2.2min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  2.2min finished


In [ ]:
score=cross_val_score(MLPClassifier(),X_train_bal,y_train_bal,cv=5,scoring='roc_auc',n_jobs=-1,verbose=2)
print(score.mean())

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


0.9601445389018501 CatBoostClassifier


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  1.5min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  1.5min finished


Selected Model :-

RandomForestClassifier

# HyperParameter tuning for RandomForestClassifier

In [ ]:
grid_param={'bootstrap': [True],
 'max_depth':np.arange(2,15),
 'max_features': ['auto'],
 'min_samples_leaf': np.arange(0,5),
 'min_samples_split': np.arange(0,5),
 'n_estimators': [200,300,400,500]
 }


In [ ]:
RF=RandomForestClassifier()

In [ ]:
rf_random = RandomizedSearchCV(RF, grid_param, n_iter = 30, cv = 3, verbose=2, random_state=42, n_jobs = -1)

In [ ]:
rf_random.fit(X_train_bal,y_train_bal)

Fitting 3 folds for each of 30 candidates, totalling 90 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done  90 out of  90 | elapsed:  3.5min finished


RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    max_samples=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
               

In [ ]:
rf_random.best_score_

0.9165307058303563

In [ ]:
rf_random.best_estimator_

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=14, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=2, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=300,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=14, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=2, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=300,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

#### RF grid search on just balanced data

In [ ]:
grid_param2={
 'max_depth':np.arange(12,16),
 'min_samples_leaf': np.arange(1,3),
 'min_samples_split': np.arange(1,3),
 'n_estimators': [300,400]
 }


In [ ]:
rf_grid = GridSearchCV(RF, grid_param2, cv = 2, verbose=2,n_jobs = -1,scoring='roc_auc')

In [ ]:
rf_grid.fit(X_train_bal,y_train_bal)

Fitting 2 folds for each of 32 candidates, totalling 64 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done  64 out of  64 | elapsed:  2.9min finished


GridSearchCV(cv=2, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                              oob_score=False,
                                              rando

In [ ]:
rf_grid.best_estimator_

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=15, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=400,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=15, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=400,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
rf_grid.best_score_

0.9816543552247932

In [ ]:
pred=rf_grid.predict(X_test1)

In [ ]:
from sklearn.metrics import confusion_matrix,accuracy_score,roc_auc_score

In [ ]:
confusion_matrix(y_test1,pred)

array([[7663,  717],
       [ 409,  706]])

In [ ]:
accuracy_score(y_test1,pred)

0.8814112690889943

In [ ]:
roc_auc_score(y_test1,pred)

0.7738114986568491

In [ ]:
model3=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=15, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=400,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

#### RF grid search on data after feature reduction PCA

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca=PCA()

In [ ]:
X_train_pca=pca.fit_transform(X_train_bal)

In [ ]:
X_test_pca=pca.transform(X_test1)

In [ ]:
model3.fit(X_train_pca,y_train_bal)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=15, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=400,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
pca_pred=model3.predict(X_test_pca)

In [ ]:
confusion_matrix(y_test1,pca_pred)

array([[7445,  935],
       [ 268,  847]])

In [ ]:
roc_auc_score(y_test1,pca_pred)

0.8240330383038839

RF grid search After Feature selection using RF

In [ ]:
model3.fit(X_train_bal_sel,y_train_bal)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=15, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=400,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
pred_sel=model3.predict(X_test_bal_sel)

In [ ]:
confusion_matrix(y_test1,pred_sel)

array([[7669,  711],
       [ 421,  694]])

In [ ]:
roc_auc_score(y_test1,pred_sel)

0.7687883279642967

# Final model

## It is observed that the best model is Random forest and it gives the best score on data after feature reduction

### Making a pipeline to transform data, apply smote on Train data, apply PCA  and then apply a Random forest model 

In [ ]:
from imblearn.pipeline import Pipeline as imbpipeline

SMOTE does not work on sklearns pipeline function

 Since, SMOTE doesn’t have a ‘fit_transform’ method, we cannot use it with ‘Scikit-Learn’ pipeline

In [ ]:
pipeline = imbpipeline(steps = [['Column_tranform',col_trans],
                              ['smote', SMOTE(random_state=45)],
                             ['PCA',PCA()],
                                ['classifier', RandomForestClassifier()]])

In [ ]:
stratified_kfold = StratifiedKFold(n_splits=3,
                                       shuffle=True,
                                       random_state=11)

In [ ]:
param_grid1={
 'classifier__max_depth':np.arange(12,16),
 'classifier__min_samples_leaf': np.arange(1,3),
 'classifier__min_samples_split': np.arange(1,3),
 'classifier__n_estimators': [300,400]
 }


In [ ]:
grid_search = GridSearchCV(estimator=pipeline,
                           param_grid=param_grid1,
                           scoring='roc_auc',
                           cv=stratified_kfold,
                           n_jobs=-1,verbose=2)

y_train1 is lable encoded

In [ ]:
grid_search.fit(X_train,y_train1)

Fitting 3 folds for each of 32 candidates, totalling 96 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  9.0min
[Parallel(n_jobs=-1)]: Done  96 out of  96 | elapsed: 26.3min finished
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


GridSearchCV(cv=StratifiedKFold(n_splits=3, random_state=11, shuffle=True),
             error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[['Column_tranform',
                                        ColumnTransformer(n_jobs=None,
                                                          remainder='passthrough',
                                                          sparse_threshold=0.3,
                                                          transformer_weights=None,
                                                          transformers=[('scale',
                                                                         Pipeline(memory=None,
                                                                                  steps=[('StandardS',
                                                                                          RobustScaler(copy=True,
                                                                              

In [ ]:
grid_search.best_estimator_

Pipeline(memory=None,
         steps=[('Column_tranform',
                 ColumnTransformer(n_jobs=None, remainder='passthrough',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('scale',
                                                  Pipeline(memory=None,
                                                           steps=[('StandardS',
                                                                   RobustScaler(copy=True,
                                                                                quantile_range=(25.0,
                                                                                                75.0),
                                                                                with_centering=True,
                                                                                with_scaling=True))],
                                         

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=15, max_features='auto',
                                        max_leaf_nodes=None, max_samples=None,
                                        min_impurity_decrease=0.0,
                                        min_impurity_split=None,
                                        min_samples_leaf=2, min_samples_split=2,
                                        min_weight_fraction_leaf=0.0,
                                        n_estimators=400, n_jobs=None,
                                        oob_score=False, random_state=None,
                                        verbose=0, warm_start=False)]]

In [ ]:
test_score = grid_search.score(X_test, y_test1)

In [ ]:
dump(grid_search, open('Final_model2.pkl', 'wb'))

In [ ]:
test_score

0.9160838853987178

In [ ]:
grid_search.predict(X_test)

array([1, 0, 0, ..., 0, 0, 1])